<a href="https://colab.research.google.com/github/kwafafoa/neqsimpython/blob/master/DewpointCalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install neqsim


     |████████████████████████████████| 30.7 MB 55.9 MB/s 
     |████████████████████████████████| 448 kB 56.1 MB/s 


In [5]:
import neqsim
from neqsim.thermo.thermoTools import*
import pandas as pd


In [10]:
inputdata = {
    'feedFlowRateTrain1': 11411.9,
    'feedPressure': 89.0,
    'feedTemperature':4.0,
    'sep1pressure': 85.0,
    'cooler1T': -7.0,
    'glycolflow': 1.0,
    'expOutPressure':67.0
  
    }

In [11]:
feedFluid = {'ComponentName':  ['water','MEG','methane','ethane','C6','C7'],
             'MolarComposition[-]':[1.0, 0.0, 0.5, 0.1, 0.1, 0.3],
             'MolarMass[kg/mol]': [None, None, None, None, 0.091, 0.19],
             'RelativeDensity[-]': [None, None, None, None, 0.7, 0.86]
}
reservoirFluidf = pd.DataFrame(feedFluid)
print(reservoirFluidf.head(20).to_string())
fluid7 = fluid_df(reservoirFluidf)

glycolFluid = fluid7.clone()
glycolFluid.setMolarComposition([0.0, 1.0, 0.0, 0.0, 0.0, 0.0])

  ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0         water                  1.0                NaN                 NaN
1           MEG                  0.0                NaN                 NaN
2       methane                  0.5                NaN                 NaN
3        ethane                  0.1                NaN                 NaN
4            C6                  0.1              0.091                0.70
5            C7                  0.3              0.190                0.86


In [22]:
from neqsim.process import clearProcess, runProcess, separator3phase, stream, valve, cooler, expander, mixer
clearProcess()
feedStream = stream(fluid7)
feedStream.setFlowRate(inputdata['feedFlowRateTrain1'], 'kg/hr')
feedStream.setPressure(inputdata['feedPressure'], 'barg')
feedStream.setTemperature(inputdata['feedTemperature'],'C')

glycolfeedStream = stream(glycolFluid)
glycolfeedStream.setPressure(inputdata['feedPressure'],'barg')
glycolfeedStream.setTemperature(inputdata['feedTemperature'],'C')
glycolfeedStream.setFlowRate(inputdata['glycolflow'],'kg/hr')




slugcatcher = separator3phase(feedStream)
gasFromslugcatcher = stream(slugcatcher.getGasOutStream())

valve1 = valve(gasFromslugcatcher)
valve1.setOutletPressure(inputdata['sep1pressure'], 'barg')

sep1 = separator3phase(valve1.getOutStream())

cooler1 = cooler(sep1.getGasOutStream())
cooler1.setOutTemperature(inputdata['cooler1T'], 'C')

sep2 = separator3phase(cooler1.getOutStream())

mixer1 = mixer()
mixer1.addStream(sep2.getGasOutStream())
mixer1.addStream(glycolfeedStream)

expander1 = expander(mixer1.getOutStream(),inputdata['expOutPressure'])

sep3 = separator3phase(expander1.getOutStream())

gasexport = stream(sep3.getGasOutStream())


feedStream2 = stream(fluid7)
























runProcess()

In [29]:
printFrame(gasexport.getFluid())


gasexport.setPressure(70.0,'bara')
print('hydrate/water dew point temperature',gasexport.getHydrateEquilibriumTemperature()-273.15)


                        total               gas                   
             water 3.17605E-6        3.17605E-6    [mole fraction]
               MEG 9.50701E-8        9.50701E-8    [mole fraction]
           methane  9.2915E-1         9.2915E-1    [mole fraction]
            ethane 7.02136E-2        7.02136E-2    [mole fraction]
             C6_PC 6.33122E-4        6.33122E-4    [mole fraction]
             C7_PC 1.81531E-8        1.81531E-8    [mole fraction]
                                                                  
           Density                    7.94322E1           [kg/m^3]
     PhaseFraction                          1E0    [mole fraction]
         MolarMass  1.70754E1         1.70754E1          [kg/kmol]
          Z factor                   7.28621E-1                [-]
Heat Capacity (Cp)                    3.52863E0          [kJ/kg*K]
Heat Capacity (Cv)                    1.67787E0          [kJ/kg*K]
    Speed of Sound                    3.71484E2            [m/